# Statefarm Data - Phase4 - Comparing VGG16, InceptionV3 and ResNet50

Comparing various models after removal of marginal quality data and using 14000 cases of pseudo labeled data

In [3]:
import theano
from theano.sandbox import cuda
cuda.use('gpu0')

Using gpu device 0: Tesla K80 (CNMeM is disabled, cuDNN 5103)
/home/ubuntu/anaconda2/lib/python2.7/site-packages/theano/sandbox/cuda/__init__.py:600: UserWarning: Your cuDNN version is more recent than the one Theano officially supports. If you see any problems, try updating Theano or downgrading cuDNN to version 5.
  warnings.warn(warn)


In [4]:
%matplotlib inline
IMPORT_DIR = '/home/ubuntu/nbs'
%cd $IMPORT_DIR

/home/ubuntu/nbs


In [5]:
from __future__ import division,print_function

import os, json
from glob import glob
import numpy as np
np.set_printoptions(precision=4, linewidth=100)
from matplotlib import pyplot as plt
import daveutils
from daveutils import *
import davenet
from davenet import *
import my_cv_modeler
from my_cv_modeler import *

Using Theano backend.


In [6]:
ALL_DATA_DIR = '/home/ubuntu/'
DATA_HOME_DIR = ALL_DATA_DIR+'statefarm/'
TRAIN_DIR = DATA_HOME_DIR+'train/'
VALID_DIR = DATA_HOME_DIR+'valid/'
SAMPLE_DIR = DATA_HOME_DIR+'sample/'
MODELS_DIR = DATA_HOME_DIR+'models/'
RESULTS_DIR = DATA_HOME_DIR+'results/'
TEST_DIR = DATA_HOME_DIR+'test/'

# 1. Prepare Data

#### Identify and remove poor quality training data

Previously Identified Data that is badly classified or multi-class:

In [7]:
bad_img_nums=np.array(['16927','101091','31121','27454','49471','47068','18737','14223','68147','68040','54867',
                  '38427', '8131', '62871', '99733', '92769','75819', '79819'])
#n.b. some of these image numbers at in the validation folder

In [8]:
%cd $DATA_HOME_DIR

/home/ubuntu/statefarm


Move bad images from /train to /bad folder

In [43]:
from shutil import move
from shutil import copytree #(src, dst, symlinks=False, ignore=None)
%cd $DATA_HOME_DIR
def move_bad_to_bad_folder(from_dir, bad_filenames, bad_dir = 'bad_train'):  #bad_dir must not already exist
    count = 0
    copytree(from_dir, bad_dir)
    g = glob(from_dir+'/c?/*.jpg')
    for filename in g:
        if filename[len(from_dir)+8:][:-4] in bad_filenames:
            print(filename[len(from_dir)+1:])
            move(filename, bad_dir+'/'+filename[len(from_dir)+1:])
            count+=1
    print(count,"items successfully moved from /",from_dir,"folder to: ../",bad_dir)

/home/ubuntu/statefarm


In [45]:
move_bad_to_bad_folder('train', bad_img_nums, 'bad_train')

c8/img_18737.jpg
c8/img_49471.jpg
c8/img_68147.jpg
c0/img_14223.jpg
c0/img_47068.jpg
c0/img_79819.jpg
c0/img_31121.jpg
c0/img_101091.jpg
c0/img_16927.jpg
c2/img_62871.jpg
c4/img_92769.jpg
c4/img_38427.jpg
c5/img_27454.jpg
13 items successfully moved from / train folder to: ../ bad_train


In [46]:
move_bad_to_bad_folder('valid', bad_img_nums, 'bad_valid')

c7/img_68040.jpg
c7/img_75819.jpg
c9/img_99733.jpg
c1/img_54867.jpg
c5/img_8131.jpg
5 items successfully moved from / valid folder to: ../ bad_valid


#### Now prepare the Filenames and Labels Arrays

In [47]:
(val_classes, trn_classes, val_labels, trn_labels, 
    val_filenames, filenames, test_filenames) = get_classes(DATA_HOME_DIR)

Found 32821 images belonging to 10 classes.
Found 3827 images belonging to 10 classes.
Found 45758 images belonging to 1 classes.


In [48]:
trn_image_nums = np.array([vfile[7:][:-4] for vfile in filenames])


In [49]:
#List of training image numbers:
trn_image_nums.shape

(32821,)

# 2. Create a Sequential Vgg Model with frozen Conv Layers

### 1. Add fc_bn layers, and train only the final layer

Import the fully trained Vgg16bn network from Imagenet

In [50]:
vgg = Dave16()
model = vgg.model
last_conv_idx = [i for i,l in enumerate(model.layers) if type(l) is Convolution2D][-1]
conv_layers = model.layers[:last_conv_idx+1]

Make it so that the convoluted layers are not trainable

In [52]:
count_frozen = 0
for layer in conv_layers:
    layer.trainable = False
    if layer.trainable == False: count_frozen+=1
print(count_frozen,"layers are frozen")    

31 layers are frozen


Create a sequential model of the convolutional layers

In [53]:
conv_model = Sequential(conv_layers)

#### Add fully connected layers and initial weights from previous training

Firstly, import pretrained fully connected model

In [54]:
top_hat_model = read_model(4, cross='old') 

In [55]:
p=0.5
def add_bn_layers(p, model):
    new_model = model
    new_model.add(MaxPooling2D(input_shape=conv_layers[-1].output_shape[1:]))
    new_model.add(Flatten())
    new_model.add(Dropout(p/2))
    new_model.add(Dense(128, activation='relu'))
    #new_model.layers[len(new_model.layers)].set_weights(top.layers[3].get_weights())
    new_model.add(BatchNormalization())
    new_model.add(Dropout(p/2))
    new_model.add(Dense(128, activation='relu'))
    #new_model.layers[len(new_model.layers)].set_weights(top.layers[6].get_weights())
    new_model.add(BatchNormalization())
    new_model.add(Dropout(p))
    new_model.add(Dense(10, activation='softmax'))  
    #new_model.layers[len(new_model.layers)].set_weights(top.layers[9].get_weights())
    return new_model

In [56]:
full_model = add_bn_layers(0.8, conv_model)

Now initialise the weights of the dense layers by copying from the previously trained fc_bn model

In [57]:
full_model.layers[last_conv_idx+3+1].set_weights(top_hat_model.layers[3].get_weights())
full_model.layers[last_conv_idx+6+1].set_weights(top_hat_model.layers[6].get_weights())
full_model.layers[last_conv_idx+9+1].set_weights(top_hat_model.layers[9].get_weights())

In [58]:
full_model.compile(Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

In [59]:
len(full_model.layers)

41

# Train the Model - including use of 14k pseudo label test cases

Use ImageGenerator because there are too many training images to store (resized) in an array.
1. Not using data augmentation at this stage.
2. Not using validation data for training at this stage.

n.b. Mixiterator was not used.  Only test data having a prediction probability >0.995 has been used.
This data is considered to be of such good quality that it can be mixed with real data. The pseudo training data will make up 43% of the training data at this stage (39% after validation data is added). Yes, it's a little high, but lets see how it goes.. 

Create the image generator (no augmentation)

In [61]:
gen = ImageDataGenerator()

In [65]:
generator = gen.flow_from_directory(
        'train',
        target_size=(224, 224),
        batch_size=64,
        class_mode='categorical',
        shuffle=True)

Found 32821 images belonging to 10 classes.


In [66]:
val_generator = gen.flow_from_directory(
        'valid',
        target_size=(224, 224),
        batch_size=64,
        class_mode='categorical',
        shuffle=True)
val_generator.N

Found 3827 images belonging to 10 classes.


3827

In [64]:
generator.N

32821

In [68]:
full_model.fit_generator(
        generator,
        samples_per_epoch=generator.N,
        nb_epoch=2,
        validation_data=val_generator,
        nb_val_samples=1000)

Epoch 1/2
32821/32821 [==============================] - 870s - loss: 0.1056 - acc: 0.9654 - val_loss: 1.0931 - val_acc: 0.7334
Epoch 2/2
32821/32821 [==============================] - 872s - loss: 0.0473 - acc: 0.9853 - val_loss: 0.7631 - val_acc: 0.8291


In [69]:
full_model.fit_generator(
        generator,
        samples_per_epoch=generator.N,
        nb_epoch=2,
        validation_data=val_generator,
        nb_val_samples=3000)

Epoch 1/2
32821/32821 [==============================] - 924s - loss: 0.0373 - acc: 0.9884 - val_loss: 0.8449 - val_acc: 0.8303
Epoch 2/2
32821/32821 [==============================] - 924s - loss: 0.0284 - acc: 0.9915 - val_loss: 0.9337 - val_acc: 0.8182


In [71]:
full_model.optimizer.lr=0.01
full_model.fit_generator(
        generator,
        samples_per_epoch=generator.N,
        nb_epoch=1,
        validation_data=val_generator,
        nb_val_samples=val_generator.N)

Epoch 1/1
32821/32821 [==============================] - 944s - loss: 0.0228 - acc: 0.9930 - val_loss: 1.0425 - val_acc: 0.7993


## Adding Data Augmentation

In [72]:
dgen = ImageDataGenerator(  rotation_range=5,
                            width_shift_range=0.1,
                            height_shift_range=0.05,
                            channel_shift_range = 20
                         )
tgenerator = dgen.flow_from_directory(
        'train',
        target_size=(224, 224),
        batch_size=64,
        class_mode='categorical',
        shuffle=True)
full_model.optimizer.lr=0.01
full_model.fit_generator(
        tgenerator,
        samples_per_epoch=100000,
        nb_epoch=1,
        validation_data=val_generator,
        nb_val_samples=val_generator.N)

Found 32821 images belonging to 10 classes.
Epoch 1/1
 99999/100000 [============================>.] - ETA: 0s - loss: 0.1497 - acc: 0.9569

/home/ubuntu/anaconda2/lib/python2.7/site-packages/keras/engine/training.py:1462: UserWarning: Epoch comprised more than `samples_per_epoch` samples, which might affect learning results. Set `samples_per_epoch` correctly to avoid this warning.
  warnings.warn('Epoch comprised more than '


100063/100000 [==============================] - 2679s - loss: 0.1497 - acc: 0.9569 - val_loss: 0.8665 - val_acc: 0.8048


In [86]:
save_model(full_model, 1, cross='vgg_minus_val')

In [103]:
full_model.optimizer.lr=0.005
full_model.fit_generator(
        tgenerator,
        samples_per_epoch=100000,
        nb_epoch=1,
        validation_data=val_generator,
        nb_val_samples=val_generator.N)

Epoch 1/1
100063/100000 [==============================] - 2681s - loss: 0.0740 - acc: 0.9781 - val_loss: 0.8735 - val_acc: 0.8247


In [104]:
save_model(full_model, 2, cross='vgg_minus_val')

In [105]:
full_model.optimizer.lr=0.005
full_model.fit_generator(
        tgenerator,
        samples_per_epoch=100000,
        nb_epoch=1,
        validation_data=val_generator,
        nb_val_samples=val_generator.N)

Epoch 1/1
100063/100000 [==============================] - 2683s - loss: 0.0623 - acc: 0.9819 - val_loss: 0.8635 - val_acc: 0.8116


In [ ]:
save_model(full_model, 3, cross='vgg_minus_val')

## Try Inception v3

In [73]:
from keras.applications.inception_v3 import InceptionV3
# create the base pre-trained model
#input_tensor = Input(shape=(3,224, 224))  # this assumes K.image_dim_ordering() == 'tf'
#base_model = InceptionV3(weights='imagenet',input_shape=(3,244,244) include_top=False)
base_model = InceptionV3(weights='imagenet', include_top=False)
# add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)
# let's add a fully-connected layer
x = Dense(1024, activation='relu')(x)
# and a logistic layer -- let's say we have 200 classes
predictions = Dense(10, activation='softmax')(x)

# this is the model we will train
model = Model(input=base_model.input, output=predictions)

# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional InceptionV3 layers
for layer in base_model.layers:
    layer.trainable = False

# compile the model (should be done *after* setting layers to non-trainable)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')

# train the model on the new data for a few epochs
model.fit_generator(
        tgenerator,
        samples_per_epoch=tgenerator.N,
        nb_epoch=1,
        validation_data=val_generator,
        nb_val_samples=val_generator.N)

86917120/86916664 [==============================] - 6s     
th
Epoch 1/1
32821/32821 [==============================] - 484s - loss: 0.8416 - val_loss: 1.8106


In [78]:
from keras.optimizers import SGD
model.compile(optimizer=SGD(lr=0.01, momentum=0.9), loss='categorical_crossentropy', metrics=['accuracy'])

In [79]:
history = model.fit_generator(
        tgenerator,
        samples_per_epoch=tgenerator.N,
        nb_epoch=1,
        validation_data=val_generator,
        nb_val_samples=val_generator.N)

Epoch 1/1
32821/32821 [==============================] - 484s - loss: 0.4395 - acc: 0.8544 - val_loss: 1.6566 - val_acc: 0.5579


In [80]:
# at this point, the top layers are well trained and we can start fine-tuning
# convolutional layers from inception V3. We will freeze the bottom N layers
# and train the remaining top layers.

# let's visualize layer names and layer indices to see how many layers
# we should freeze:
for i, layer in enumerate(base_model.layers):
    print(i, layer.name)

0 input_1
1 convolution2d_14
2 batchnormalization_5
3 convolution2d_15
4 batchnormalization_6
5 convolution2d_16
6 batchnormalization_7
7 maxpooling2d_7
8 convolution2d_17
9 batchnormalization_8
10 convolution2d_18
11 batchnormalization_9
12 maxpooling2d_8
13 convolution2d_22
14 batchnormalization_13
15 convolution2d_20
16 convolution2d_23
17 batchnormalization_11
18 batchnormalization_14
19 averagepooling2d_1
20 convolution2d_19
21 convolution2d_21
22 convolution2d_24
23 convolution2d_25
24 batchnormalization_10
25 batchnormalization_12
26 batchnormalization_15
27 batchnormalization_16
28 mixed0
29 convolution2d_29
30 batchnormalization_20
31 convolution2d_27
32 convolution2d_30
33 batchnormalization_18
34 batchnormalization_21
35 averagepooling2d_2
36 convolution2d_26
37 convolution2d_28
38 convolution2d_31
39 convolution2d_32
40 batchnormalization_17
41 batchnormalization_19
42 batchnormalization_22
43 batchnormalization_23
44 mixed1
45 convolution2d_36
46 batchnormalization_27
47 c

In [81]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, 3, None, None) 0                                            
____________________________________________________________________________________________________
convolution2d_14 (Convolution2D) (None, 32, None, None)0           input_1[0][0]                    
____________________________________________________________________________________________________
batchnormalization_5 (BatchNormal(None, 32, None, None)0           convolution2d_14[0][0]           
____________________________________________________________________________________________________
convolution2d_15 (Convolution2D) (None, 32, None, None)0           batchnormalization_5[0][0]       
___________________________________________________________________________________________

dense_8 (Dense)                  (None, 10)            10250       dense_7[0][0]                    
Total params: 2108426
____________________________________________________________________________________________________


In [83]:
# we chose to train the top 2 inception blocks, i.e. we will freeze
# the first 172 layers and unfreeze the rest:
for layer in model.layers[:172]:
    layer.trainable = False
for layer in model.layers[172:]:
    layer.trainable = True

# we need to recompile the model for these modifications to take effect
# we use SGD with a low learning rate
model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), loss='categorical_crossentropy', metrics=['accuracy'])

# we train our model again (this time fine-tuning the top 2 inception blocks
# alongside the top Dense layers
history = model.fit_generator(
        tgenerator,
        samples_per_epoch=tgenerator.N,
        nb_epoch=1,
        validation_data=val_generator,
        nb_val_samples=val_generator.N)

Epoch 1/1
32821/32821 [==============================] - 530s - loss: 0.3063 - acc: 0.8986 - val_loss: 1.4870 - val_acc: 0.6044


In [84]:
history = model.fit_generator(
        tgenerator,
        samples_per_epoch=tgenerator.N,
        nb_epoch=3,
        validation_data=val_generator,
        nb_val_samples=val_generator.N)

Epoch 1/3
32821/32821 [==============================] - 530s - loss: 0.2256 - acc: 0.9277 - val_loss: 1.4025 - val_acc: 0.6216
Epoch 2/3
32821/32821 [==============================] - 530s - loss: 0.1901 - acc: 0.9392 - val_loss: 1.4101 - val_acc: 0.6347
Epoch 3/3
32821/32821 [==============================] - 529s - loss: 0.1630 - acc: 0.9486 - val_loss: 1.3689 - val_acc: 0.6376


Save the model!!!

In [85]:
save_model(model, 1, cross='Incep')

## Now try ResNet50

In [88]:
from keras.applications.resnet50 import ResNet50
from keras.preprocessing import image
from keras.applications.resnet50 import preprocess_input, decode_predictions

res_base_model = ResNet50(include_top=False, weights='imagenet')#,input_shape=(3,224,224)(include_top=True, weights='imagenet', input_tensor=None, input_shape=None))

94511104/94653016 [============================>.] - ETA: 0s

In [89]:
for layer in res_base_model.layers:
    layer.trainable = False
# add a global spatial average pooling layer
rx = res_base_model.output
rx = GlobalAveragePooling2D()(rx)
# let's add a fully-connected layer
rx = Dense(1024, activation='relu')(rx)
# and a logistic layer -- let's say we have 200 classes
predictions = Dense(10, activation='softmax')(rx)

# this is the model we will train
res_model = Model(input=res_base_model.input, output=predictions)

# compile the model (should be done *after* setting layers to non-trainable)
res_model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])


th


In [91]:
# train the model on the new data for a few epochs
res_model.fit_generator(
        tgenerator,
        samples_per_epoch=tgenerator.N,
        nb_epoch=3,
        validation_data=val_generator,
        nb_val_samples=val_generator.N)

Epoch 1/3
32821/32821 [==============================] - 571s - loss: 0.4219 - acc: 0.8600 - val_loss: 1.3356 - val_acc: 0.6389
Epoch 2/3
32821/32821 [==============================] - 570s - loss: 0.2633 - acc: 0.9133 - val_loss: 1.2056 - val_acc: 0.6935
Epoch 3/3
32821/32821 [==============================] - 570s - loss: 0.2000 - acc: 0.9353 - val_loss: 1.7014 - val_acc: 0.6083


#### Next, train the last few layers. First, view the model

In [92]:
res_model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_2 (InputLayer)             (None, 3, None, None) 0                                            
____________________________________________________________________________________________________
zeropadding2d_14 (ZeroPadding2D) (None, 3, None, None) 0           input_2[0][0]                    
____________________________________________________________________________________________________
conv1 (Convolution2D)            (None, 64, None, None)0           zeropadding2d_14[0][0]           
____________________________________________________________________________________________________
bn_conv1 (BatchNormalization)    (None, 64, None, None)0           conv1[0][0]                      
___________________________________________________________________________________________

dense_9 (Dense)                  (None, 1024)          2098176     globalaveragepooling2d_2[0][0]   
____________________________________________________________________________________________________
dense_10 (Dense)                 (None, 10)            10250       dense_9[0][0]                    
Total params: 2108426
____________________________________________________________________________________________________


In [93]:
for i, layer in enumerate(res_base_model.layers):
    print(i, layer.name)

0 input_2
1 zeropadding2d_14
2 conv1
3 bn_conv1
4 activation_1
5 maxpooling2d_10
6 res2a_branch2a
7 bn2a_branch2a
8 activation_2
9 res2a_branch2b
10 bn2a_branch2b
11 activation_3
12 res2a_branch2c
13 res2a_branch1
14 bn2a_branch2c
15 bn2a_branch1
16 merge_3
17 activation_4
18 res2b_branch2a
19 bn2b_branch2a
20 activation_5
21 res2b_branch2b
22 bn2b_branch2b
23 activation_6
24 res2b_branch2c
25 bn2b_branch2c
26 merge_4
27 activation_7
28 res2c_branch2a
29 bn2c_branch2a
30 activation_8
31 res2c_branch2b
32 bn2c_branch2b
33 activation_9
34 res2c_branch2c
35 bn2c_branch2c
36 merge_5
37 activation_10
38 res3a_branch2a
39 bn3a_branch2a
40 activation_11
41 res3a_branch2b
42 bn3a_branch2b
43 activation_12
44 res3a_branch2c
45 res3a_branch1
46 bn3a_branch2c
47 bn3a_branch1
48 merge_6
49 activation_13
50 res3b_branch2a
51 bn3b_branch2a
52 activation_14
53 res3b_branch2b
54 bn3b_branch2b
55 activation_15
56 res3b_branch2c
57 bn3b_branch2c
58 merge_7
59 activation_16
60 res3c_branch2a
61 bn3c_bran

In [95]:
for layer in res_model.layers[162:]:
    layer.trainable = True

# we need to recompile the model for these modifications to take effect
# we use SGD with a low learning rate
res_model.compile(optimizer=SGD(lr=0.01, momentum=0.9), loss='categorical_crossentropy', metrics=['accuracy'])

# we train our model again (this time fine-tuning the top 2 inception blocks
# alongside the top Dense layers
history = res_model.fit_generator(
        tgenerator,
        samples_per_epoch=tgenerator.N,
        nb_epoch=1,
        validation_data=val_generator,
        nb_val_samples=val_generator.N)

Epoch 1/1
32821/32821 [==============================] - 616s - loss: 0.0313 - acc: 0.9907 - val_loss: 1.1335 - val_acc: 0.7484


In [96]:
# we use SGD with a low learning rate
res_model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), loss='categorical_crossentropy', metrics=['accuracy'])

history = res_model.fit_generator(
        tgenerator,
        samples_per_epoch=tgenerator.N,
        nb_epoch=1,
        validation_data=val_generator,
        nb_val_samples=val_generator.N)

Epoch 1/1
32821/32821 [==============================] - 614s - loss: 0.0206 - acc: 0.9939 - val_loss: 0.9452 - val_acc: 0.7711


In [101]:
res_model.compile(optimizer=SGD(lr=0.00001, momentum=0.9), loss='categorical_crossentropy', metrics=['accuracy'])
history = res_model.fit_generator(
        tgenerator,
        samples_per_epoch=tgenerator.N,
        nb_epoch=2,
        validation_data=val_generator,
        nb_val_samples=val_generator.N)

Epoch 1/2
32821/32821 [==============================] - 615s - loss: 0.0176 - acc: 0.9949 - val_loss: 0.8937 - val_acc: 0.7771
Epoch 2/2
32821/32821 [==============================] - 615s - loss: 0.0144 - acc: 0.9961 - val_loss: 0.9069 - val_acc: 0.7779


Save the Model

In [102]:
save_model(res_model, 1, cross='ResNet')

In [107]:
res_model.compile(Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])
history = res_model.fit_generator(
        tgenerator,
        samples_per_epoch=tgenerator.N*2,
        nb_epoch=1,
        validation_data=val_generator,
        nb_val_samples=val_generator.N)

Epoch 1/1
65642/65642 [==============================] - 1195s - loss: 0.0697 - acc: 0.9790 - val_loss: 1.1419 - val_acc: 0.7638


## Use All Training Data

Next, include the validation data into the training data via the MixIterator

Only bother doing this for the winner out of Vgg, ResNet50 and Inception

In [100]:
#mi = MixIterator([tgenerator, val_generator])
# Doesn't work because 'DirectoryIterator' object does not support indexing. Only works when the X_train 
# and X_valid images are already stored in numpy arrays.

Next step: include the validation images in the model.  See ..phase5.  Use data in statefarm1 because the validation data was not split off.  Use Vgg model because it is better than ResNet50 so far.  